# ЯПдляИИ
### Пиняева Анна Андреевна
### НПИмд-01-24
### 1132249572

In [8]:
import tensorflow as tf

In [9]:
print(tf.__version__)

2.16.2


In [10]:
import keras

In [11]:
print(keras.__version__)

3.5.0


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

dataset_path = "/Users/anna/Downloads/CLEVR_v1.0/images"
clevr_dataset = image_dataset_from_directory(dataset_path, image_size=(128, 128), batch_size=32)

Found 100000 files belonging to 3 classes.


In [4]:
import json

# Путь к аннотациям
path_to_annotations = '/Users/anna/Downloads/CLEVR_v1.0/scenes/CLEVR_train_scenes.json'

# Загрузка данных
with open(path_to_annotations, 'r') as f:
    clevr_data = json.load(f)

# Структура данных
scenes = clevr_data['scenes']  
print(scenes[0]) 


{'image_index': 0, 'objects': [{'color': 'blue', 'size': 'large', 'rotation': 269.8517172617167, 'shape': 'cube', '3d_coords': [-1.3705521821975708, 2.0794010162353516, 0.699999988079071], 'material': 'rubber', 'pixel_coords': [269, 88, 12.661545753479004]}, {'color': 'green', 'size': 'large', 'rotation': 292.2219458666971, 'shape': 'cylinder', '3d_coords': [-2.9289753437042236, -1.7488206624984741, 0.699999988079071], 'material': 'metal', 'pixel_coords': [93, 108, 11.522202491760254]}, {'color': 'cyan', 'size': 'small', 'rotation': 25.545135239473026, 'shape': 'cube', '3d_coords': [1.5515961647033691, 0.6776641607284546, 0.3499999940395355], 'material': 'rubber', 'pixel_coords': [319, 162, 10.045343399047852]}, {'color': 'brown', 'size': 'large', 'rotation': 327.3489188814305, 'shape': 'cylinder', '3d_coords': [-0.25301405787467957, -2.3089325428009033, 0.699999988079071], 'material': 'metal', 'pixel_coords': [132, 159, 9.392304420471191]}, {'color': 'gray', 'size': 'small', 'rotation

In [6]:
# Функция для создания пар объектов и меток (левее/правее)
def create_relational_data_from_scenes(scenes):
    object_pairs = []  # Список для хранения пар объектов
    labels = [] # Список для хранения меток (1 - левее, 0 - правее)
    
    for scene in scenes:
        objects = scene['objects'] # Список объектов в сцене
        for i in range(len(objects)):  # Первый объект
            for j in range(i + 1, len(objects)): # Второй объект
                obj1 = objects[i]
                obj2 = objects[j]
                
                # Определение метки: если координата x первого объекта меньше, он левее второго
                label = 1 if obj1['3d_coords'][0] < obj2['3d_coords'][0] else 0
                
                # Извлекаем координаты объектов (x, y, z)
                obj1_features = np.array(obj1['3d_coords'])
                obj2_features = np.array(obj2['3d_coords'])
                object_pairs.append((obj1_features, obj2_features))
                labels.append(label)
    
    return np.array(object_pairs), np.array(labels)

In [8]:
import numpy as np
from sklearn.model_selection import train_test_split
# Генерация данных
object_pairs, labels = create_relational_data_from_scenes(scenes)

# Преобразование данных в формат для модели
object_pairs = object_pairs.reshape((object_pairs.shape[0], 2, 3))  # 2 объекта, каждый описан 3D координатами

# Разделение данных на обучающую и тестовую выборки
train_pairs, test_pairs, train_labels, test_labels = train_test_split(object_pairs, labels, test_size=0.2)

In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Функция для построения encoder слоя трансформера
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Multi-Head Attention
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs
    
    # Feed Forward Network
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return layers.LayerNormalization(epsilon=1e-6)(x + res)

# Функция для построения трансформера
def build_transformer_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0, mlp_dropout=0):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)
    
    x = layers.GlobalAveragePooling1D()(x)
    
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    
    outputs = layers.Dense(1, activation="sigmoid")(x)  # Для бинарной классификации отношений между объектами
    return keras.Model(inputs, outputs)

# Параметры модели
input_shape = (2, 3)  # 2 объекта, каждый описан 3 координатами (x, y, z)
model = build_transformer_model(
    input_shape=input_shape,
    head_size=64,
    num_heads=4,
    ff_dim=128,
    num_transformer_blocks=4,
    mlp_units=[128],
    dropout=0.1,
    mlp_dropout=0.1
)

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Вывод структуры модели
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 2, 3)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 2, 3)      │      3,843 │ input_layer[0][0… │
│ (MultiHeadAttentio… │                   │            │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 2, 3)      │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 2, 3)      │          6 │ dropout_1[0][0]   │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 2, 3)      │          0 │ layer_normalizat… │
│                     │                   │            │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 2, 128)    │        512 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 2, 128)    │          0 │ conv1d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 2, 3)      │        387 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 2, 3)      │          0 │ conv1d_1[0][0],   │
│                     │                   │            │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 2, 3)      │          6 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 2, 3)      │      3,843 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 2, 3)      │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 2, 3)      │          6 │ dropout_4[0][0]   │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 2, 3)      │          0 │ layer_normalizat… │
│                     │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 2, 128)    │        512 │ add_2[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 2, 128)    │          0 │ conv1d_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 2, 3)      │        387 │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 2, 3)      │          0 │ conv1d_3[0][0],   │
│                     │                   │            │ add_2[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 2, 3)      │          6 │ add_3[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 19,657 (76.79 KB)

 Trainable params: 19,657 (76.79 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
# Обучение модели
history = model.fit(
    train_pairs, 
    train_labels, 
    epochs=10, 
    batch_size=32, 
    validation_data=(test_pairs, test_labels)
)


Epoch 1/10
35987/35987 ━━━━━━━━━━━━━━━━━━━━ 436s 12ms/step - accuracy: 0.5096 - loss: 0.6929 - val_accuracy: 0.5352 - val_loss: 0.6905
Epoch 2/10
35987/35987 ━━━━━━━━━━━━━━━━━━━━ 421s 12ms/step - accuracy: 0.5117 - loss: 0.6925 - val_accuracy: 0.5015 - val_loss: 0.6932
Epoch 3/10
35987/35987 ━━━━━━━━━━━━━━━━━━━━ 430s 12ms/step - accuracy: 0.5007 - loss: 0.6931 - val_accuracy: 0.5238 - val_loss: 0.6921
Epoch 4/10
35987/35987 ━━━━━━━━━━━━━━━━━━━━ 430s 12ms/step - accuracy: 0.5207 - loss: 0.6923 - val_accuracy: 0.5179 - val_loss: 0.6923
Epoch 5/10
35987/35987 ━━━━━━━━━━━━━━━━━━━━ 423s 12ms/step - accuracy: 0.5190 - loss: 0.6924 - val_accuracy: 0.5154 - val_loss: 0.6926
Epoch 6/10
35987/35987 ━━━━━━━━━━━━━━━━━━━━ 433s 12ms/step - accuracy: 0.5156 - loss: 0.6926 - val_accuracy: 0.5149 - val_loss: 0.6926
Epoch 7/10
35987/35987 ━━━━━━━━━━━━━━━━━━━━ 422s 12ms/step - accuracy: 0.5125 - loss: 0.6927 - val_accuracy: 0.4985 - val_loss: 0.6932
Epoch 8/10
35987/35987 ━━━━━━━━━━━━━━━━━━━━ 421s 12ms/s

In [11]:
# Оценка модели на тестовых данных
test_loss, test_accuracy = model.evaluate(test_pairs, test_labels)
print(f"Test Accuracy: {test_accuracy:.4f}")


8997/8997 ━━━━━━━━━━━━━━━━━━━━ 33s 4ms/step - accuracy: 0.4991 - loss: 0.6931
Test Accuracy: 0.4985


In [12]:
# Предсказания модели на нескольких тестовых примерах
predictions = model.predict(test_pairs[:5])
print("Predictions:", predictions)  # Предсказанные вероятности (левее/правее)
print("True labels:", test_labels[:5])  # Истинные метки


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 852ms/step
Predictions: [[0.4999634]
 [0.4999634]
 [0.4999634]
 [0.4999634]
 [0.4999634]]
True labels: [1 1 0 0 1]
